# Abstract words

In [1]:
# import all functions for this chapter/notebook
from abslithist.words import *

## Semantic fields

### Save word norms from psycholinguistic sources

These are downloaded from sources.

In [2]:
source(gen_orignorms)

def gen_orignorms():
    # init
    fields=defaultdict(set)
    norms=[]

    # add fields (so far, only quant/scale-based ones)
    funcs=[
        gen_norms_paivio, # Paivo et al
        gen_norms_mrc,    # MRC
        gen_norms_brys,   # Brysbaert et al
        gen_norms_lsn     # LSN
    ]

    # run through functions
    for func in tqdm(funcs,desc='Building fields and norms from sources'):
        func(norms)

    # save norms
    qdf=pd.DataFrame(norms)
    qdf=qdf.drop_duplicates(['word','source'],keep='first')
    qdf.to_csv(PATH_NORMS,index=False)

In [3]:
gen_orignorms()

Building fields and norms from sources: 100%|██████████| 4/4 [00:01<00:00,  2.86it/s]


### Divide high/low words into "fields"

This is done automatically by the z-score cutoff function in *get_fields_from_norms()*.

In [4]:
# Show the relevant functions
source(get_origfields)
source(get_fields_from_norms)

def get_origfields():
    return get_fields_from_norms(PATH_NORMS)

def get_fields_from_norms(path_norms,zcut=ZCUT,name_neither='Neither'):
    dfnorms=pd.read_csv(path_norms)
    fields=dict()
    for source,sourcedf in dfnorms.groupby('source'):
        if not '.' in source: continue
        contrast,method=source.split('.')
        if not '-' in contrast: continue
        pos,neg=contrast.split('-')
        fields[f'{contrast}.{method}.{pos}']=poswords=set(sourcedf[sourcedf.z<=-zcut].word)
        fields[f'{contrast}.{method}.{neg}']=negwords=set(sourcedf[sourcedf.z>=zcut].word)
        fields[f'{contrast}.{method}.{name_neither}'] = set(sourcedf.word) - poswords - negwords
    for field in fields:
        fields[field]={w for w in fields[field] if type(w)==str}
    return fields

In [5]:
# Show fields
fields = get_origfields()
show_fields(fields)

,num,words
field,,
Abs-Conc.LSN-Sens.Neither,25055,"everyday, spearhead, forethoughtfully, memoranda, fawning, brazenness, mouse, exhilaration, savi..."
Abs-Conc.LSN-Perc.Neither,24781,"mock, jeeringly, shed, vibrantly, atmosphere, nonattached, nonparticipant, pest, scalability, un..."
Abs-Conc.LSN-Imag.Neither,24051,"drunk, incisor, stockbreeder, fictionally, lovably, originally, kazoo, viselike, multivitamin, c..."
Abs-Conc.MT-Conc.Neither,22674,"preindustrial, editing, unincorporated, whipped, priesthood, codeword, homogeneous, semitranspar..."
Abs-Conc.MT-Conc.Conc,7662,"aorta, salsa, stethoscope, nun, rubberstamp, carriageway, rockstar, surfboarding, sleet, salon ...."
Abs-Conc.MT-Conc.Abs,6721,"definitional, unknowable, gratuitousness, or, bode, psychopathological, impassion, incitingly, i..."
Abs-Conc.LSN-Imag.Conc,6525,"discount, looks, nurse, limbless, slugger, wellmannered, beanbag, structure, human, nonmetal ......"
Abs-Conc.LSN-Imag.Abs,6232,"unashamedly, yet, cheerless, unproduced, unrighteousness, incriminate, hypo, discerningly, super..."
Abs-Conc.LSN-Perc.Abs,6071,"imposition, intending, consumptive, meaningless, unsoundness, irreversible, slung, quantificatio..."


In [6]:
show_contrasts(fields)

neg  \
contrast method                                                        
Abs-Conc LSN-Imag  hibiscus, image, faraway, withering, spammer, ...   
         LSN-Perc  burn, thorniness, flavorless, donkey, explaini...   
         LSN-Sens  reins, porn, creamer, sausage, napkin, dream, ...   
         MRC-Conc  ark, minister, rabbi, harp, nchina, farm, cord...   
         MRC-Imag  goblet, beetle, pet, albatross, soup, mouth, h...   
         MT-Conc   gauge, spaceship, cornmeal, gemstone, crab, sa...   
         PAV-Conc  basement, geese, soil, fireplace, string, skin...   
         PAV-Imag  restaurant, hotel, grass, skin, salad, suds, p...   

                                                                 pos  \
contrast method                                                        
Abs-Conc LSN-Imag  nonparty, mythical, breeze, impassion, singson...   
         LSN-Perc  irreverently, ready, repeal, friskily, implici...   
         LSN-Sens  impassion, supercharger, obliging, dandily, un...   
         MRC-Conc  vague, relinquish, came, drab, when, anticipat...   
         MRC-Imag  caul, result, method, without, fact, plausible...   
         MT-Conc   languishingly, offend, recallable, if, concept...   
         PAV-Conc  loyalty, greed, agreement, situation, essence,...   
         PAV-Imag  trouble, necessity, answer, misconception, pro...   

                                                             neither  
contrast method                                                       
Abs-Conc LSN-Imag  routine, throat, bonus, irresolvable, sordidne...  
         LSN-Perc  cellular, spray, scalability, anal, strung, wo...  
         LSN-Sens  outlaw, cadaver, gong, goods, veteran, metabol...  
         MRC-Conc  silence, oat, sit, bid, farewell, interior, re...  
         MRC-Imag  ghost, reading, subtraction, yelp, cloak, howl...  
         MT-Conc   gluttonousness, neurobiology, modernism, depic...  
         PAV-Conc  debacle, church, sky, errand, queen, abode, im...  
         PAV-Imag  string, affection, panorama, honeycomb, lark, ...

## Semantic vectors

### First compute them

That is, find the word-to-field cosine distances. Just store them in the data/models directory as csv's (```word2field_dists.csv```).

In [ ]:
source(compute_vecfields)

In [ ]:
# compute_vecfields()

### Generate vector-based norms

Using the word2field distances generate above, loop over each distance table (corresponding to a model run), standardize all distances (zscore) within models, and then save the standardized distances in a manner similar to how the original word norms were saved.

In [ ]:
source(gen_vecnorms)

In [ ]:
# gen_vecnorms()

### Aggregate vector norms by period

The separate sets of norms for each model-period are aggregated by period. Currently this is a century: across all models within C17, C18, C19, and C20, a given word's (z-)score along a given vector contrast is averaged (median'd). These are then saved with a new period column  into a single *data.wordnorms_vec.csv* in *data/fields*.

#### Generate aggregated data

In [ ]:
from abslithist.words.fields import agg_vecnorms
agg_vecnorms??

In [ ]:
# Run?
# agg_vecnorms()

#### Get/inspect aggregated data

In [ ]:
# Testing result
from abslithist.words.fields import get_vecnorms
get_vecnorms??

In [ ]:
# Run?
# get_vecnorms()

## Divide all norms into fields